## Code Example (S&P 500 option)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize 
import datetime as dtt
from datetime import datetime as dt
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
import plotly.graph_objects as go
from plotly.graph_objs import Surface
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objects as go
from plotly.graph_objs import Surface
from plotly.offline import iplot, init_notebook_mode
import scipy.stats as st
from dateutil import parser
import csv

### Import the experation dates (file Experation_dates.csv)

In [2]:
Experation_dates =list(pd.read_csv("Experation_dates.csv")["0"])

### Calculate the different maturities 

In [3]:
maturities = []
for i in range(0,23):
     maturities.append(((parser.parse(Experation_dates[i]) - parser.parse(Experation_dates[0])).days +1)/365.25)

### Import the data from the attached folder 
### get the common strikes  beteween different maturities
### get the prices for those strikes and maturities 

In [4]:
data = {}
for i in range(0,23):
    data[i] = pd.read_csv(f'Data/d{i}.csv')

In [5]:
option_prices = {}
for i in range(0,23):
    option_prices[Experation_dates[i]] = {}
    option_prices[Experation_dates[i]] ['Strike']= data[i]['Strike']
    option_prices[Experation_dates[i]] ['Prices']= (data[i]['Ask']+data[i]['Bid'])/2

In [6]:
all_strikes = [v[1]['Strike'] for v in option_prices.items()]
common_strikes = set.intersection(*map(set,all_strikes))
common_strikes = sorted(common_strikes)

In [7]:
prices = []
for v in option_prices.items():
    price = [v[1]['Prices'][i] for i,x in enumerate(v[1]['Strike']) if x in common_strikes]
    prices.append(price)

In [8]:
price_arr = np.array(prices, dtype=object)
Market_prices = pd.DataFrame(price_arr, index = maturities, columns = common_strikes)

### Get the rates for different maturities 

In [9]:
yield_maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20])
yeilds = np.array([0.15,0.27,0.50,0.93,1.52,2.13,2.32,2.34,2.37,2.32,2.65,2.52]).astype(float)/100

### Use the nelson_siegel_svensson to fit the model

In [10]:
curve_fit, status = calibrate_nss_ols(yield_maturities,yeilds) 

### The COS method for option pricing and the characteristic function for the Heston model

In [11]:
def COSM(cf,CP,S0,r,tau,K,N,L):
    K = np.array(K).reshape([len(K),1])  # reshape K to a column vector
    i = complex(0.0,1.0) 
    x0 = np.log(S0 / K)   
    k = np.linspace(0,N-1,N).reshape([N,1])  
    a = -L * np.sqrt(tau)
    b = L * np.sqrt(tau)
    u = k * np.pi / (b - a) 
    Psi = lambda c, d: 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) * (np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  
                - np.cos(k * np.pi * (c - a) / (b - a)) * np.exp(c)
                + k * np.pi / (b - a) * np.sin(k * np.pi * (d - a) / (b - a))  
                - k * np.pi / (b - a) * np.sin(k * np.pi * (c - a) / (b - a)) * np.exp(c))
    Phi = lambda c,d: np.sin(k * np.pi * (d - a) / (b - a))  - np.sin(k * np.pi *  (c - a)/(b - a))
    H_k = np.linspace(0,0,len(k)).reshape([len(k),1]) 
    
    # H_k for call and put 
    if str(CP).upper()=="C":                  
            H_k[1:] = 2.0 / (b - a) * (Psi(0,b)[1:] - Phi(0,b)[1:] * (b - a) / (k[1:] * np.pi)) 
            H_k[0] = 2.0 / (b - a) * (Psi(0,b)[0] - b) 
    elif str(CP).upper()=="P":
            H_k[1:] = 2.0 / (b - a) * (Phi(a,0)[1:]* (b - a) / (k[1:] * np.pi)     - Psi(a,0)[1:])  
            H_k[0] = 2.0 / (b - a) * (-a - Psi(a,0)[0]) 
    mat = np.exp(i * np.outer(- a , u) + i*np.outer(x0,u)) # "iux" is included here
    temp = cf(u) * H_k 
    temp[0] = 0.5 * temp[0]    
    value = np.exp(-r * tau) * K * np.real(mat.dot(temp))     
    return value
def ChFHestonModel(r,tau,kappa,gamma,vbar,v0,rho):
    i = complex(0.0,1.0)
    D1 = lambda u: np.sqrt(np.power(kappa-gamma*rho*i*u,2)+(u*u+i*u)*gamma*gamma)
    g = lambda u: (kappa-gamma*rho*i*u-D1(u))/(kappa-gamma*rho*i*u+D1(u))
    C = lambda u: (1.0-np.exp(-D1(u)*tau))/(gamma*gamma*(1.0-g(u)*np.exp(-D1(u)*tau)))*(kappa-gamma*rho*i*u-D1(u))
    # Note that we exclude the term -r*tau, as the discounting is performed in the COS method
    A = lambda u: r * i*u *tau + kappa*vbar*tau/gamma/gamma *(kappa*gamma*rho*i*u-D1(u)) - 2*kappa*vbar/gamma/gamma*np.log((1.0-g(u)*np.exp(-
    D1(u)*tau))/(1.0-g(u)))
    # Characteristic function for the Heston model
    cf = lambda u: np.exp(A(u) + C(u)*v0)
    return cf

### The error function (objective function)

In [12]:
#maturities = Market_data.index
#maturities= maturities.tolist()
S0 = 3855
N = 128
L = 10
CP = 'C'
K = common_strikes
r= curve_fit(np.array(maturities))
tau = np.array(maturities)
w = len(K)*len(maturities)
def SqErr(x):
    "x is a list of paramters to be optimized "
    v0, kappa, vbar, gamma, rho = [param for param in x]
    lis = []
    for i in range(0,len(maturities)):    
        cf = ChFHestonModel(r[i],tau[i],kappa,gamma,vbar,v0,rho)
        pr = COSM(cf,CP,S0,r[i],tau[i],K,N,L).reshape(len(K))
        pp = pr.tolist()
        lis.append(pp)
    COShestonprices = pd.DataFrame(lis, index = maturities, columns=K)

    error = ( (Market_prices-COShestonprices)**2 / w ).values.sum()
          
    return error

### Heston prices using the COS method from some given parameters

In [13]:
kappa = 1.5
gamma = 0.5
rho = -0.5
v0 = 0.017
vbar = 0.03
lis = []
for i in range(0,len(maturities)):    
    cf = ChFHestonModel(r[i],tau[i],kappa,gamma,vbar,v0,rho)
    pr = COSM(cf,CP,S0,r[i],tau[i],K,N,L).reshape(len(K))
    pp = pr.tolist()
    lis.append(pp)
COShestonprices = pd.DataFrame(lis, index = maturities, columns=K)

### Defining the constraints 

In [14]:
params = {"v0": {"x0": 0.01, "lbub": [1e-3,0.1]}, 
          "kappa": {"x0": 0.5, "lbub": [1e-3,5]},
          "vbar": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "gamma": {"x0": 0.4, "lbub": [1e-2,1]},
          "rho": {"x0": -0.7, "lbub": [-1,-0.000000005]}
          #"lambd": {"x0": 0.03, "lbub": [-1,1]},
          }
x0 = [param["x0"] for key, param in params.items()]
bounds = [param["lbub"] for key, param in params.items()]

### Minimizing our objective function

In [15]:
result = minimize(SqErr, x0, tol = 1e-3, method='SLSQP', bounds=bounds)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



### Calculate the option prices using the optimized parameters

In [16]:
v0, kappa, vbar, gamma, rho= [param for param in result.x]
lis = []
for i in range(0,len(maturities)):    
    cf = ChFHestonModel(r[i],tau[i],kappa,gamma,vbar,v0,rho)
    pr = COSM(cf,CP,S0,r[i],tau[i],K,N,L).reshape(len(K))
    pp = pr.tolist()
    lis.append(pp)
Hestoncalibprices = pd.DataFrame(lis, index = maturities, columns=K)

### Show the resutls 

In [17]:
Market_pricesLong = Market_prices.melt(ignore_index=False).reset_index()
Market_pricesLong.columns = ['Maturity', 'Strike', 'Price']
Hestoncalibpriceslong = Hestoncalibprices.melt(ignore_index=False).reset_index()
Hestoncalibpriceslong.columns = ['Maturity', 'Strike', 'Price']
Market_pricesLong['hestonprice'] = Hestoncalibpriceslong['Price']

In [19]:
#init_notebook_mode()
fig = go.Figure(data=[go.Mesh3d(x=Market_pricesLong.Maturity, y=Market_pricesLong.Strike, z=Market_pricesLong.Price, color='mediumblue', opacity=0.5)])
fig.add_scatter3d(x=Market_pricesLong.Maturity, y=Market_pricesLong.Strike, z=Market_pricesLong.hestonprice, mode='markers')
fig.update_layout(
    title_text='Market Prices (Mesh) vs Calibrated Heston Prices (Markers)',
    scene = dict(xaxis_title='Maturities',
                    yaxis_title='Strike ',
                    zaxis_title='Option price'),
    height=700,
    width=700
)
fig.show()